In [1]:
from tensorflow.keras.models import load_model
import cv2
import mediapipe as mp
import numpy as np

model = load_model("yoga_pose_model.h5")



pose_labels = ["Downdog", "Goddess", "Plank", "Tree", "Warrior2"]


/Users/gurparkashsingh/Desktop/Machine Learning/Image Classification/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/gurparkashsingh/Desktop/Machine Learning/Image Classification/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/gurparkashsingh/Desktop/Machine Learning/Image Classification/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
print(model.input_shape)


(None, 99)


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# --- Load your trained model ---
model = tf.keras.models.load_model("yoga_pose_model.h5")
pose_labels = ["Tree", "Warrior", "Chair", "Cobra", "DownwardDog", "Plank", "Mountain"]

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils


def calculate_angle(a, b, c):
    """Calculate angle between 3 points"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    return 360-angle if angle > 180 else angle


cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # --- Extract landmark coords (for model input) ---
        keypoints = []
        for lm in landmarks:
            keypoints.extend([lm.x, lm.y, lm.z])
        input_data = np.array(keypoints).reshape(1, -1)

        # --- Predict yoga pose ---
        prediction = model.predict(input_data, verbose=0)
        class_id = np.argmax(prediction)
        confidence = np.max(prediction)
        pose_name = pose_labels[class_id]

        # --- Compute angles ---
        l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * w,
                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h]
        l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * w,
                   landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * h]
        l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * w,
                   landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * h]

        r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * w,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * h]
        r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * w,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * h]
        r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * w,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * h]

        left_elbow_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
        right_elbow_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)

        # --- Show results ---
        cv2.putText(image, f"Pose: {pose_name} ({confidence:.2f})", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        cv2.putText(image, f"L_Elbow: {int(left_elbow_angle)}", 
                    tuple(np.array(l_elbow, dtype=int)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

        cv2.putText(image, f"R_Elbow: {int(right_elbow_angle)}", 
                    tuple(np.array(r_elbow, dtype=int)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow("Yoga Pose Recognition", image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1755712777.528041 4090537 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1755712777.582301 4091420 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755712777.589847 4091427 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
